# DECISION TREE IMPLEMENTATION


#  Printed the Decision tree steps as specified in the example below.

# Building the actual decision tree as well

In [1]:
#importing the necessary libraries
import numpy as np
import pandas as pd
from sklearn import datasets
import math

In [2]:
#loading the dataset from sklearn
iris=datasets.load_iris()

In [3]:
#creating a dataframe using Pandas
df=pd.DataFrame(iris.data)

In [4]:
#renaming the columns
df.columns=["sl","sw","pl","pw"]

In [5]:
#writing a function to find the label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val<boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Now writing a function to convert continuous data into labelled data
#there are total 4 labels that is a,b,c,d

def tolabel(df,old_feature_name):
    second=df[old_feature_name].mean()
    minimum=df[old_feature_name].min()
    first=(minimum + second)/2
    maximum=df[old_feature_name].max()
    third=(maximum + second)/2
    return df[old_feature_name].apply(label, args=(first,second,third))

In [6]:
#Now converting all columns to labelled data
df['sl_labelled'] = tolabel(df, 'sl')
df['sw_labelled'] = tolabel(df, 'sw')
df['pl_labelled'] = tolabel(df, 'pl')
df['pw_labelled'] = tolabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labelled,sw_labelled,pl_labelled,pw_labelled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [7]:
df.drop(['sl','sw','pl','pw'], axis=1, inplace=True)

In [8]:
set(df['sl_labelled'])

{'a', 'b', 'c', 'd'}

# Now Building the actual decision tree

In [18]:
class TreeNode:
    def __init__(self, feature_to_split, current_output):
        """" Init Block : 
            __init__() : Initializes the tree node
        Parameters :
            feature_to_split : feature upon which the node is to be split 
                              and it is None for leaf node
            current_output   : Class with current majority at this instance to be predicted
    """
        self.feature_to_split = feature_to_split
        
        self.children = {} 
        """ Dictionary :
                    Key   : Feature upon which the node is to split
                    Value : Child Node
        """
        
        self.current_output = current_output
        
        self.index=-1
        
    def add_child(self,feature_name,child):
        """" Function : 
            add_child() : Adds a child node to the Tree
        Parameters :
            feature_name : Feature upon which splitting is done
            child        : Reference of child node   
    """
        self.children[feature_name] = child

# Now calculating the gain ratio

In [20]:
#here making a function
def gain(x, y, selected_feature):
    """" Function : 
            gain() : Returns the gain ratio
        Parameters :
            x                : Training Data
            y                : Target classes 
            selected_feature : The feature whose gain ratio is to be calculated
    """
    
    #Finding the entropy of the node before splitting
    original_entropy = entropy(y)
    
    # Assuming the entropy after splitting to be 0 initially
    entropy_after_splitting = 0 
    
    # diff_values_feature represents the different unique values of the best_feature selected
    diff_values_feature = set(x[:,selected_feature])
    df = pd.DataFrame(x)
    
    # Appending Y values as the last column in the newely created dataframe 
    df[df.shape[1]] = y
    starting_size = df.shape[0] 
    
    # Assuming the split_info to be 0 intially
    split_info = 0
    
    # Calculating the entropy and split_info after splitting
    for i in diff_values_feature:
        df1 = df[df[selected_feature] == i]
        current_size = df1.shape[0]
        entropy_after_splitting += (current_size/starting_size)*entropy(df1[df1.shape[1]-1])
        split_info += (-current_size/starting_size)*math.log2(current_size/starting_size)
        
    # Checking if the split_info is 0, if it is 0 then it will give ZeroDivisionException so changing it
    if split_info == 0 :
        return -math.inf 
        #returning negative infinity when split info is 0
    
    # Finding the info_gain in order to find the gain_ratio
    info_gain = original_entropy - entropy_after_splitting
    
    # Finding the gain_ratio
    gain_ratio = info_gain / split_info
    
    return gain_ratio

# Now counting the frequency of each class in target

In [21]:
#making a function
def class_counter(y):
    """" Function : 
            class_counter() : Returns a dictionary with keys as unique values of y(i.e no of classes) 
                              and the corresponding value as its frequency
        Parameters :
            y         : Target classes 
    """
   
    # Creating a dicitonary to store the count of different classes in y
    classes_count = {}
    
    # Traversing y (target classes) to count the frequncy of each class
    for i in y:
        # Checking if it is present in the dictiory if yes then increasing its value by 1 else storing its value as 1
        if i in classes_count:
            classes_count[i]+=1
        else:
            classes_count[i]=1
            
    return classes_count

# Now calculating the entropy

In [22]:
def entropy(y):
    """" Function : 
            entropy() : Returns the entropy of any node
        Parameters :
            y         : Target classes 
    """
    # Finding the count of different classes at current level
    classes_count = class_counter(y)
    
    # Assuming the entropy to be 0 intially
    ent = 0
    
    # Finding the total size of the data
    total_data_points = len(y)
    
    # Calculating the entropy
    for class_ in classes_count:
        prob = classes_count[class_]/total_data_points
        ent += (-prob)*math.log2(prob)
    
    return ent

# Decision tree function

In [25]:
#making a fucntion for the same
def decision_tree(x, y, features_index, level, classes, features_names = np.array([i for i in df.columns])):
    """" Function : 
            decision_tree() : Decision Tree Classifier
        Parameters :
            x              : Training Data
            y              : Training Target
            features_index : Index Values of different features available
            level          : Current Depth Level of Tree
            classes        : Different Classes of target
            feature_names  : Names of different Features available - default
    """
    ###############
    # BASE CASES #
    ##############
    
    # 1. If unused feature is empty i.e no feature is left to split upon
    #    In this case we will return the majority
    
    if len(features_index) == 0:
        classes_count = class_counter(y)
        
        # Printing the current level
        print("Level",level)
        
        # Finding the count of different classes at current level
        for i in classes:
            print("Count of",i,"=",classes_count.get(i,0))
            
        # Finding the entropy at the current level
        print("Current Entropy  is =",entropy(y))
        print("Reached leaf Node")
        print()
        
        max_count = max(classes_count.values())
        for class_ in classes_count:
            if classes_count[class_]==max_count:
                output = class_
        return TreeNode(None,output)

    # 2. If target contains only one distinct value i.e a pure class
    #    In this case no further split is possible 
    
    if len(set(y)) == 1:
        
        # Finding the count of classes at this level
        for class_ in classes:
            if class_ in y:
                output = class_
                print("Count of",class_,"=",len(y))
            else :
                print("Count of",class_,"=",0)
        
        # The entropy of pure node is 0
        print("Current Entropy is =  0.0")
        print("Reached leaf Node")
        print()
        return TreeNode(None,output)
    
    
    #########################################################
    # FINDING THE BEST FEATURE TO SPLIT BASES ON GAIN RATIO #
    #########################################################
    
    # Assuming a deafult value for the best_feature
    best_feature = None
    # Assuming the max_gain to be -infinity
    max_gain = -math.inf
    
    # Traversing through each feature and finding the gain ratio
    for f in features_index:
        # Finding the Gain Ratio of current feature
        gain_ratio = gain(x, y, f)
        
        # Comparing the gain ratio to the max_gain and accordingly assigning the values of best_feature
        if gain_ratio > max_gain:
            max_gain = gain_ratio
            best_feature = f
    
    # Printing the current level of node
    print("Level",level)
    
    # Finding the count of different classes in target
    classes_count = class_counter(y)
    
    # Printing the count of classes at current level
    for i in classes:
        print("Count of",i,"=",classes_count.get(i,0))
    
    # Finding the entropy at the current level
    print("Current Entropy is =",entropy(y))
    
    # Splitting based on the best_feature and printing the max_gain
    print("Splitting on feature" , features_names[best_feature] , "with gain ratio" , max_gain)
    print()
    
    max_count = max(classes_count.values())
    for class_ in classes_count:
        if classes_count[class_]==max_count:
            output = class_
                
    current_node = TreeNode(features_names[best_feature],output)
    
    # diff_values_feature represents the different unique values of the best_feature selected
    diff_values_feature = set(x[:,best_feature])
    
    # Creating a data frame of the x values
    df = pd.DataFrame(x)
    
    # appending y values at the end of the data frame.
    df[df.shape[1]] = y
    
    # removing the selected feature from the list
    features_index.remove(best_feature)
    
    
    #####################
    # CALLING RECURSION #
    #####################
    
    # Creating the nodes for each unique values for the selected feature and calling recursion 
    for i in diff_values_feature:
        
        # Creating a new dataframe with value of selected feature = i
        next_df = df[df[best_feature] == i]
        
        # recursively calling on the splits
        new_node = decision_tree(next_df.iloc[:,0:next_df.shape[1]-1].values,next_df.iloc[:,next_df.shape[1]-1].values,features_index,level+1,classes)
        current_node.add_child(i,new_node)
    return current_node

# Look at the main programm

In [26]:
# Getting the target data as y
y = iris.target

# Getting the training data as x
x = df.values

# Storing the different unique classes in the target
classes = set(y)

# Setting the intial level as 0
level = 0

# Finding the index array for features
features_index = [i for i in range(len(x[0]))]

# Calling the decision_tree fucntion and storing the root of the tree implemented
root = decision_tree(x, y, features_index, level, classes)

Level 0
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 1.584962500721156
Splitting on feature pw_labelled with gain ratio 0.6996382036222091

Level 1
Count of 0 = 0
Count of 1 = 40
Count of 2 = 16
Current Entropy is = 0.863120568566631
Splitting on feature pl_labelled with gain ratio 0.4334099495621067

Level 2
Count of 0 = 0
Count of 1 = 39
Count of 2 = 8
Current Entropy is = 0.6581912658132185
Splitting on feature sl_labelled with gain ratio 0.12674503775809332

Level 3
Count of 0 = 0
Count of 1 = 23
Count of 2 = 7
Current Entropy is = 0.783776947484701
Splitting on feature sw_labelled with gain ratio 0.07092036405148876

Level 4
Count of 0 = 0
Count of 1 = 6
Count of 2 = 0
Current Entropy  is = 0.0
Reached leaf Node

Level 4
Count of 0 = 0
Count of 1 = 3
Count of 2 = 1
Current Entropy  is = 0.8112781244591328
Reached leaf Node

Level 4
Count of 0 = 0
Count of 1 = 14
Count of 2 = 6
Current Entropy  is = 0.8812908992306927
Reached leaf Node

Level 3
Count of 0 = 

# Printing the actual decision tree

In [28]:
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

C:\Users\m\anaconda3\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


# Now creating DOT_Data file

In [31]:
import pydotplus
from collections import deque
def export_tree_pdf(root):
        # returns the tree as dot data
        # if filename is specified the function 
        # will save the pdf file in current directory which consists of the visual reresentation of the tree
        dot_data = '''digraph Tree {
node [shape=box] ;'''
        queue = deque()
        r = root
        queue.append(r)
        count = 0
        if r.index == -1:
            r.index = count
        dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(count,r.feature_to_split,r.current_output) 
        # Doing LEVEL ORDER traversal in the tree (using a queue)
        while len(queue) != 0 :
            node = queue.popleft()
            for i in node.children:
                count+=1
                if(node.children[i].index==-1):
                    node.children[i].index = count
                # Creating child node
                dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(node.children[i].index,node.children[i].feature_to_split,node.children[i].current_output) 
                # Connecting parent node with child
                dot_data = dot_data + "\n{} -> {} [ headlabel=\"Feature value = {}\"]; ".format(node.index,node.children[i].index,i)
                # Adding child node to queue
                queue.append(node.children[i])
        dot_data = dot_data + "\n}"
        return dot_data

# Printing image(png) of actual tree


In [32]:
dot_data = export_tree_pdf(root)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.set_size("10000,50!")
graph.write_pdf("actualtree.pdf")
graph.write_png("actualtree.png")
Image(graph.create_png())